In [1]:
from PandaStation import PandaStation
from PandaStation import AddShape
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG



import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory
from RRT import PandaRRTPlanner, PandaRRTompl
from collections import OrderedDict

In [2]:
poses = OrderedDict()
times = OrderedDict()

start = RigidTransform(
  R=RotationMatrix([
    [0.955336489125606, -4.69354560239685e-13, 0.29552020666133966],
    [2.3666020818845225e-12, -1.0, -9.238812855303034e-12],
    [0.2955202066613397, 9.525553773197387e-12, -0.955336489125606],
  ]),
  p=[0.5772631662903251, -1.0560725542374789e-12, 0.7451829336930107],
)

poses['start'] = start
times['start'] = 0

prepick = RigidTransform(
  R=RotationMatrix([
    [6.123233995736766e-17, 1.0, -1.2246467991473532e-16],
    [1.0, -6.123233995736766e-17, 7.498798913309288e-33],
    [0.0, -1.2246467991473532e-16, -1.0],
  ]),
  p=[-0.09999999999999998, -0.65, 0.29],
)


prepick = RigidTransform(
  R=RotationMatrix([
    [0.00032634849740877943, -0.9999996551926252, 0.0007635794522856558],
    [-0.9999996551926253, -0.00032634849740877943, 2.491930928292587e-07],
    [0.0, -0.000763579715573573, -0.999999416946018],
  ]),
  p=[-0.0894124211702011, -0.6675974424332297, 0.14612277568551174],
)


poses['prepick'] = prepick
times['prepick'] = times['start'] + 10

pick = RigidTransform(
  R=RotationMatrix([
    [6.123233995736766e-17, 1.0, -1.2246467991473532e-16],
    [1.0, -6.123233995736766e-17, 7.498798913309288e-33],
    [0.0, -1.2246467991473532e-16, -1.0],
  ]),
  p=[-0.09999999999999999, -0.65, 0.21],
)

poses['pick_start'] = pick
poses['pick_end'] = pick
times['pick_start'] = times['prepick'] + 2
times['pick_end'] = times['pick_start'] + 2

postpick = RigidTransform(
  R=RotationMatrix([
    [6.123233995736766e-17, 1.0, -1.2246467991473532e-16],
    [1.0, -6.123233995736766e-17, 7.498798913309288e-33],
    [0.0, -1.2246467991473532e-16, -1.0],
  ]),
  p=[-0.09999999999999998, -0.65, 0.29],
)

poses['postpick'] = postpick
times['postpick'] = times['pick_end'] + 2

preplace = RigidTransform(
  R=RotationMatrix([
    [-1.0, 1.2246467991473532e-16, -1.4997597826618576e-32],
    [1.2246467991473532e-16, 1.0, -1.2246467991473532e-16],
    [0.0, -1.2246467991473532e-16, -1.0],
  ]),
  p=[0.65, 0.1, 0.35],
)

poses['preplace'] = preplace
times['preplace'] = times['postpick'] + 10

place = RigidTransform(
  R=RotationMatrix([
    [-1.0, 1.2246467991473532e-16, -1.4997597826618576e-32],
    [1.2246467991473532e-16, 1.0, -1.2246467991473532e-16],
    [0.0, -1.2246467991473532e-16, -1.0],
  ]),
  p=[0.65, 0.1, 0.29],
)

poses['place'] = place
times['place'] = times['preplace'] + 2

postplace = RigidTransform(
  R=RotationMatrix([
    [0.955336489125606, -4.69354560239685e-13, 0.29552020666133966],
    [2.3666020818845225e-12, -1.0, -9.238812855303034e-12],
    [0.2955202066613397, 9.525553773197387e-12, -0.955336489125606],
  ]),
  p=[0.5772631662903251, -1.0560725542374789e-12, 0.7451829336930107],
)

poses['poseplace'] = postplace
times['postplace'] = times['place'] + 2


opened = np.array([0.08])
closed = np.array([0])
traj_hand_command = pydrake.trajectories.PiecewisePolynomial.FirstOrderHold(
        [times['start'], times['pick_start']], np.hstack([[opened], [opened]])) # go to prepick
traj_hand_command.AppendFirstOrderSegment(times['pick_end'], closed) # picking
traj_hand_command.AppendFirstOrderSegment(times['preplace'], closed) # go to preplace
traj_hand_command.AppendFirstOrderSegment(times['place'], opened) # placing

In [3]:
class PrinterSystem(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        
        self.input_port = self.DeclareVectorInputPort('input_port', BasicVector(7))
        
        self.DeclarePeriodicEvent(0.1, 0, 
                        event=PublishEvent(
                            trigger_type=TriggerType.kPeriodic,
                            callback=self.MyPublish)) 
        
    def MyPublish(self, context, event):
        #if (context.get_time() > 1):
        #    return
        q = self.EvalVectorInput(context, self.input_port.get_index()).get_value()
        time = context.get_time()
        print(f"time: {time}, command q: {q}")

In [4]:
X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.1, -.65, 0.091]),
       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.65, 0.1, 0.09])}

builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation())
station.SetupBinStation()
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    X_O["initial"])


plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")
#c1 = AddShape(plant, Cylinder(0.03, 1), "c1")
#plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, -0.25, 0.5])) # add a model to test collisions
#c2 = AddShape(plant, Cylinder(0.03, 1), "c2")
#plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c2", c2), RigidTransform([-0.3, 0, 0.5])) # add a model to test collisions


station.Finalize()


planner = builder.AddSystem(BinPointCloudToGraspSystem())
builder.Connect(station.GetOutputPort("camera0_point_cloud"), planner.GetInputPort("camera0_pcd"))
builder.Connect(station.GetOutputPort("camera1_point_cloud"), planner.GetInputPort("camera1_pcd"))
builder.Connect(station.GetOutputPort("camera2_point_cloud"), planner.GetInputPort("camera2_pcd"))
builder.Connect(station.GetOutputPort("panda_position_measured"), planner.GetInputPort("panda_position"))

builder.Connect(planner.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
builder.Connect(planner.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))

#printer = builder.AddSystem(PrinterSystem())
#builder.Connect(planner.GetOutputPort("panda_position_command"), printer.GetInputPort("input_port"))

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))

'''
trajectory = None
avoid_names= ['bin0', 'bin1']#, 'c1', 'c2']
poses_list = list(poses.items())
times_list = list(times.items())
for i in range(len(poses_list[:2])-1):
    start = poses_list[i][1]
    end = poses_list[i+1][1]
    s_time = times_list[i][1]
    end_time = times_list[i+1][1]
    print('times:', s_time, end_time)
    print("starting rrt")
    rrt = PandaRRTompl(plant, scene_graph, plant_context, scene_graph_context, panda,
                      start, s_time, end, end_time, avoid_names)
    print("done rrt")
    if trajectory is None:
        trajectory = rrt.get_trajectory()
    else:
        trajectory.ConcatenateInTime(rrt.get_trajectory())
        
        
q_traj_system = builder.AddSystem(TrajectorySource(trajectory))
builder.Connect(q_traj_system.get_output_port(), station.GetInputPort("panda_position"))

hand_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(traj_hand_command))
hand_source.set_name("hand_command")
builder.Connect(hand_source.get_output_port(), station.GetInputPort("hand_position"))
'''

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url, role = Role.kProximity)# <- this commented part allows visualization of the collisions







diagram = builder.Build()

#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
#station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation



meshcat.start_recording()

simulator.AdvanceTo(36)#times_list[-1][1])

meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6030...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7030/static/
Connected to meshcat-server.
looking for candidate grasp
point choice number 1
trying angle 0
ik duration: 0.026325225830078125
failed ik
point choice number 2
trying angle 0
ik duration: 0.0076618194580078125
failed ik
point choice number 3
point choice number 4
trying angle 0
ik duration: 0.005557537078857422
failed ik
point choice number 5
trying angle 0
ik duration: 0.00471806526184082
failed ik
point choice number 6
trying angle 0
ik duration: 0.00414586067199707
failed ik
point choice number 7
trying angle 0
ik duration: 0.0037856101989746094
failed ik
point choice number 8
trying angle 0
ik duration: 0.0034203529357910156
failed ik
point choice number 9
trying angle 0
ik duration: 0.0032358169555664062
failed ik
point choice number 10
trying angle 0
ik duration: 0.0031807422637939453
failed ik
point choice number 11
point choice

AssertionError: could not find valid grasp pose

In [ ]:
#file = open("media/videos/ompl_pick_and_place.html", "w")
#file.write(meshcat.vis.static_html())
#file.close()